---
title: "Annotation Tables"
aliases:
    - programmatic_access/em_py_03_annotation_tables.html
format: 
    html:
        toc: true 
        code-fold: false
execute:
    eval: False
    warning: False
jupyter: python3
bibliography: references.bib
---

{{< include _annotation_tables/_version_tip.qmd >}}

The `minnie65_public` data release includes a number of annotation tables that help label the dataset.
This section describes the content of each of these tables — [see here for instructions for how to query and filter tables](/quickstart_notebooks/em_py_02_cave_quickstart.html).


Unless otherwise specificied (i.e. via `desired_resolution`), all positions are in units of 4,4,40 nm/voxel resolution.


## Common Fields

Several fields (or column names) are common to many tables.
These fall into two main classes: the spatial point columns that are how we assign annotations to cells via points in the 3d space and book-keeping columns, that are used internally to track the state of the data.

### Spatial Point Columns

Most tables have one or more **Bound Spatial Points**, which is a location in the 3d space that tells the annotation to remain associated with the root id at that location.

Bound spatial points have will have one prefix, usually `pt` (i.e. "point") and three associated columns with different suffixes: `_position`, `_supervoxel_id`, and `_root_id`.

For a given prefix `{pt}`, the three columns are as follows:

* The `{pt}_position` indicates the location of the point in 3d space.
* The `{pt}_supervoxel_id` indicates a unique identifier in the segmentation, and is mostly internal bookkeeping.
* The `{pt}_root_id` indicates the root id of the annotation at that location.

### Book-keeping Columns

Several columns are common to many or all tables, and mostly used as internal book-keeping.
Rather than describe these for every table, they will just be mentioned briefly here:

| Column | Description |
| :--- | :--- |
| `id` | A unique ID specific to the annotation within that table |
| `created` | Internal bookkeeping column, should always be `t` for data you can download |
| `valid` | A unique ID specific to the annotation within that table |
| `target_id` | Some tables reference other tables, particularly the nucleus table. If present, this column will be the same as `id` |
| `created_ref` / `valid_ref` / `id_ref` (optional) | For reference tables, the data shows both the created/valid/id of the reference annotation and the target annotation. The values with the `_ref` suffix are those of the reference table (usually something like proofreading state or cell type) and the values without a suffix ar ethose of the target table (usually a nucleus) |

: Common columns {.light .hover}


## Synapse Table
{{< include _annotation_tables/_synapses_pni_2.qmd >}}


## Nucleus tables

The 'nucleus centroid' of a cell is unlikely to change with proofreading, and so is a useful static identifier for a given cell. The results of automatic nucleus segmentation and neuron-detection are avialable in the following tables. These tables are often the 'reference' table for other annotations.

### Nucleus Detection Table
{{< include _annotation_tables/_nucleus_detection_v0.qmd >}}

### Nucleus brain area assignment
{{< include _annotation_tables/_nucleus_functional_area_assignment.qmd >}}

:::: {.callout-note appearance="minimal" collapse=true}
### Neuron-Nucleus Table

::: {.callout-important}
This table is superseded by other cell typing below, but remains here for convenient reference as it is relevant to other tables such as `aibs_metamodel_celltypes_v661`.
:::

{{< include _annotation_tables/_nucleus_ref_neuron_svm.qmd >}}

::::

## Cell Type Tables

There are several tables that contain information about the cell type of neurons in the dataset, with each table representing a different method of doing the classificaiton.
Because each method requires a different kind of information, not all cells are present in all tables.
Each of the cell types tables has the same format and in all cases the `id` column references the nucleus id of the cell in question.

### Manual Cell Types (V1 Column)

{{< include _annotation_tables/_allen_v1_column_types_slanted_ref.qmd >}}


::: {.callout-note appearance="minimal" collapse=true}
### Manual Cell Types (non-neurons)
| Cell Type | Subclass | Description |
| :--- | :--- | :--- |
| `OPC` | Non-neuronal | Oligodendrocyte precursor cell | 
| `astrocyte` | Non-neuronal | Astrocyte | 
| `microglia` | Non-neuronal | Microglia | 
| `pericyte` | Non-neuronal | Pericyte | 
| `oligo` | Non-neuronal | Oligodendrocyte | 

: AIBS Manual Cell Type definitions (non-neurons) {.light .hover}

:::

```python
# Standard query
client.materialize.query_table('aibs_column_nonneuronal_ref')

# Content-aware query
client.materialize.tables.aibs_column_nonneuronal_ref(id=example_nucleus_id).query()
```

### Predictions from soma/nucleus features

{{< include _annotation_tables/_aibs_metamodel_celltypes_v661.qmd >}}

Previous versions of this table include: `aibs_soma_nuc_metamodel_preds_v117` (run on a subset of data, the V1 column) and `aibs_soma_nuc_exc_mtype_preds_v117` (using training data labeled by another classifier: see `mtypes` below). 

### Coarse prediction from spine detection

{{< include _annotation_tables/_baylor_log_reg_cell_type_coarse_v1.qmd >}}

### Fine prediction from dendritic features

{{< include _annotation_tables/_aibs_metamodel_mtypes_v661_v2.qmd >}}

Previous versions of this table include: `aibs_soma_nuc_exc_mtype_preds_v117` (run at v117), `allen_column_mtypes_v1` and `allen_column_mtypes_v2` (run on a subset of data, the V1 column)

## Proofreading Tables

### Proofreading Status and Strategy
{{< include _annotation_tables/_proofreading_status_and_strategy.qmd >}}


:::: {.callout-note appearance="minimal" collapse=true}
#### Proofreading status at public release

::: {.callout-important}
This table is out-of-date, and remains here for convenient reference
:::

{{< include _annotation_tables/_proofreading_status_public_release.qmd >}}

::::

## Functional Coregistration Tables

To relate the structural data to functional data, cell bodies must be coregistered between the functional imaging and EM volumes. As with cell-typing, there are __manual__ and __automated__ methods for doing this, with the former having higher accuracy but lower throughput.

* `coregistration_manual_v4` : The results of manually verified coregistration. This table is well-verified, but contains fewer `ROI`s (N=15,352 root ids, 19,181 ROIs).
* `coregistration_auto_phase3_fwd_apl_vess_combined_v2` : The results of automated functional matching between the EM and 2-p functional data. This table is not manually verified, but contains more `ROI`s (N=35,466 root ids, N=83,046 ROIs).

### Manual coregistration
{{< include _annotation_tables/_coregistration_manual_v4.qmd >}}

### Automated coregistration
{{< include _annotation_tables/_coregistration_auto_phase3_fwd_apl_vess_combined_v2.qmd >}}


### Functional properties

{{< include _annotation_tables/_digital_twin_properties_bcm_coreg_v4.qmd >}}

**Merged to Automatic Coregistration**

Table name: `digital_twin_properties_bcm_coreg_auto_phase3_fwd_v2`

Follows the same scheme as `digital_twin_properties_bcm_coreg_v4`, but uses the coregistration information from: `coregistration_auto_phase3_fwd_v2`.

```python
# Standard query
client.materialize.query_table('digital_twin_properties_bcm_coreg_auto_phase3_fwd_v2')

# Content-aware query
client.materialize.tables.digital_twin_properties_bcm_coreg_auto_phase3_fwd_v2(id=example_nucleus_id).query()
```

Table name: `digital_twin_properties_bcm_coreg_apl_vess_fwd`

Follows the same scheme as `digital_twin_properties_bcm_coreg_v4`, but uses the coregistration information from: `apl_functional_coreg_vess_fwd`.

```python
# Standard query
client.materialize.query_table('digital_twin_properties_bcm_coreg_apl_vess_fwd')

# Content-aware query
client.materialize.tables.digital_twin_properties_bcm_coreg_apl_vess_fwd(id=example_nucleus_id).query()
```


## Overview of relevant tables
| Table Name | Number of Annotations | Description |
| :--- | :--- | :--- |
| `synapses_pni_2` | 337,312,429 | The locations of synapses and the segment ids of the pre and post-synaptic automated synapse detection
| `nucleus_detection_v0` | 144,120 | The locations of nuclei detected via a fully automated method |
| `nucleus_alternative_points`| 8,388 | A reference annotation table marking alternative segment_id lookup locations for a subset of nuclei in nucleus_detection_v0 that is more accurate than the centroid location listed there |
| `nucleus_ref_neuron_svm` | 144,120 |  reference annotation indicating the output of a model detecting which nucleus detections are neurons versus which are not 1 |
| `coregistration_manual_v4` | 19,181 |  A table indicating the association between individual units in the functional imaging data and nuclei in the structural data, derived from human powered matching. Includes residual and separation scores to help assess confidence |
| `coregistration_auto_phase3_fwd_apl_vess_combined_v2` | 83,046 |  A table indicating the association between individual units in the functional imaging data and nuclei in the structural data, derived from the automated procedure. Includes residuals and separation scores to help assess confidence |
| `proofreading_status_and_strategy` | 2020 |  A table indicating which neurons have been proofread on their axons or dendrites |
| `aibs_column_nonneuronal_ref` | 542 | Cell type reference annotations from a human expert of non-neuronal cells located amongst the Minnie Column |
| `allen_v1_column_types_slanted_ref` | 1,357 | Neuron cell type reference annotations from human experts of neuronal cells located amongst the Minnie Column |
| `allen_column_mtypes_v1` | 1,357 | Neuron cell type reference annotations from data driven unsupervised clustering of neuronal cells |
| `aibs_metamodel_mtypes_v661_v2`| 72,158| Reference annotations indicating the output of a model predicting cell types across the dataset based on the labels from allen_column_mtypes_v1.1 |
| `aibs_metamodel_celltypes_v661` | 94,014 | Reference annotations indicating the output of a model predicting cell classes based on the labels from allen_v1_column_types_slanted_ref and aibs_column_nonneuronal_ref |
| `baylor_log_reg_cell_type_coarse_v1` | 55,063 | Reference annotations indicated the output of a logistic regression model predicting whether the nucleus is part of an excitatory or inhibitory cell |
| `baylor_gnn_cell_type_fine_model_v2` | 49,051 | Reference annotations indicated the output of a graph neural network model predicting the cell type based on the human labels in allen_v1_column_types_slanted_ref |
| `vortex_astrocyte_proofreading_status` | 126 |  This table reports the status of a manually selected subset of astrocytes within the VISP column. Astrocyte seelection and proofreading performed as part of VORTEX.  |

: heard you like tables--here's a table for your tables {.light .hover}